In [1]:
!pip install -q transformers einops accelerate bitsandbytes
!pip install -q langchain langchain_community langchain-huggingface langchainhub langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.9 MB/s eta 0:00:00

In [2]:
import torch
import os
import getpass

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [3]:
os.environ['HF_TOKEN'] = getpass.getpass()

··········


**Loading LLM via pipeline**

In [4]:
model_id = "mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated"
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = quantization_config, token=os.environ['HF_TOKEN'])
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])

pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500,
    do_sample = True,
    repetition_penalty = 1.1,
    return_full_text = False
)
llm = HuggingFacePipeline(pipeline = pipe)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

**Template and Chain**

In [5]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a helpful virtual assistant.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{question}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [7]:
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nYou are a helpful virtual assistant.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{question}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [8]:
chain = prompt_template | llm

chain.invoke({"question": "What day is today?"})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'I\'m happy to help! However, I don\'t have the ability to know the current date or time. I\'m a large language model, I don\'t have access to real-time information or external data. But I can suggest ways for you to find out what day it is!\n\nIf you\'re looking for the current date and time, you can try checking your device\'s clock or calendar app. Alternatively, you can search online for "what day is today" or check a reliable news source or website that provides this information.\n\nIs there anything else I can help you with?'

**Prompt for RAG**

In [9]:
template_rag = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a helpful virtual assistant answering general questions.
Use the following bits of retrieved context to answer the question.
If you don't know the answer, just say you don't know. Keep your answer concise.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Question: {question}
Context: {context}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [10]:
prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)

input_variables=['context', 'question'] input_types={} partial_variables={} template="\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nYou are a helpful virtual assistant answering general questions.\nUse the following bits of retrieved context to answer the question.\nIf you don't know the answer, just say you don't know. Keep your answer concise.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nQuestion: {question}\nContext: {context}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n"


**Defining the context**

In [11]:
from datetime import date

day = date.today()
print(day)

2024-12-29


In [12]:
context = "You know that today is {}".format(day)
print(context)

You know that today is 2024-12-29


**Chain Creation/Generation**

In [13]:
chain_rag = prompt_rag | llm | StrOutputParser()

question = "What day is today? Return the date in format mm/dd/yyyy"

res = chain_rag.invoke({"question": question, "context": context})
res

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"I don't have that information. I'm not aware of the current date or any specific date outside of the provided context."

**RAG - Exploring**

In [14]:
prompt_rag

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nYou are a helpful virtual assistant answering general questions.\nUse the following bits of retrieved context to answer the question.\nIf you don't know the answer, just say you don't know. Keep your answer concise.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nQuestion: {question}\nContext: {context}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n")

In [15]:
chain_rag = prompt_rag | llm | StrOutputParser()

context = """Quarterly revenue numbers:
1: $42476.40
2: $46212.97
3: $41324.56
4: $56430.24"""

# question = "What is the revenue for the second quarter?"
question = "Which quarter had the highest revenue?"

chain_rag.invoke({"context": context, "question": question})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Based on the provided data, I can help you with that.\n\nThe highest revenue is in Quarter 4, which is $56430.24.'

**Debugging**

In [16]:
from langchain.globals import set_debug
set_debug(True)

In [17]:
question = "Which quarter had the lowest revenue?"

chain_rag.invoke({"context": context, "question": question})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "context": "Quarterly revenue numbers:\n1: $42476.40\n2: $46212.97\n3: $41324.56\n4: $56430.24",
  "question": "Which quarter had the lowest revenue?"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "context": "Quarterly revenue numbers:\n1: $42476.40\n2: $46212.97\n3: $41324.56\n4: $56430.24",
  "question": "Which quarter had the lowest revenue?"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nYou are a helpful virtual assistant answering general questions.\nUse the following bits of retrieved context to answer the question.\nIf you don't know the answer, just say you don't know. Keep your answer concise.\n<|eot_id|>\n<|start_h

'Based on the provided data, I can help you with that.\n\nThe lowest revenue is in Quarter 3, which has a value of $41324.56.'

In [18]:
set_debug(False)

**Application of RAG with larger contexts**

**Indexing Steps**

**1- Loading the content**

In [19]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [20]:
loader = WebBaseLoader(web_paths = ("https://www.bbc.com/news/entertainment-arts-68530499",),)
docs = loader.load()

In [21]:
docs

[Document(metadata={'source': 'https://www.bbc.com/news/entertainment-arts-68530499', 'title': 'The full list of winners and nominees at the Oscars 2024', 'description': "See who has won and been nominated for this year's coveted Academy Awards in Hollywood.", 'language': 'en-GB'}, page_content="The full list of winners and nominees at the Oscars 2024Skip to contentBritish Broadcasting CorporationHomeNewsSportBusinessInnovationCultureArtsTravelEarthVideoLiveHomeNewsIsrael-Gaza WarWar in UkraineUS & CanadaUKUK PoliticsEnglandN. IrelandN. Ireland PoliticsScotlandScotland PoliticsWalesWales PoliticsAfricaAsiaChinaIndiaAustraliaEuropeLatin AmericaMiddle EastIn PicturesBBC InDepthBBC VerifySportBusinessExecutive LoungeTechnology of BusinessFuture of BusinessInnovationTechnologyScience & HealthArtificial IntelligenceAI v the MindCultureFilm & TVMusicArt & DesignStyleBooksEntertainment NewsArtsArts in MotionTravelDestinationsAfricaAntarcticaAsiaAustralia and PacificCaribbean & BermudaCentral 

In [22]:
print(docs[0].page_content)

The full list of winners and nominees at the Oscars 2024Skip to contentBritish Broadcasting CorporationHomeNewsSportBusinessInnovationCultureArtsTravelEarthVideoLiveHomeNewsIsrael-Gaza WarWar in UkraineUS & CanadaUKUK PoliticsEnglandN. IrelandN. Ireland PoliticsScotlandScotland PoliticsWalesWales PoliticsAfricaAsiaChinaIndiaAustraliaEuropeLatin AmericaMiddle EastIn PicturesBBC InDepthBBC VerifySportBusinessExecutive LoungeTechnology of BusinessFuture of BusinessInnovationTechnologyScience & HealthArtificial IntelligenceAI v the MindCultureFilm & TVMusicArt & DesignStyleBooksEntertainment NewsArtsArts in MotionTravelDestinationsAfricaAntarcticaAsiaAustralia and PacificCaribbean & BermudaCentral AmericaEuropeMiddle EastNorth AmericaSouth AmericaWorld’s TableCulture & ExperiencesAdventuresThe SpeciaListEarthNatural WondersWeather & ScienceClimate SolutionsSustainable BusinessGreen LivingVideoLiveLive NewsLive SportHomeNewsSportBusinessInnovationCultureArtsTravelEarthVideoLiveAudioWeatherN

In [23]:
len(docs[0].page_content)

6076

**2 - Split**

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, add_start_index = True)
splits = text_splitter.split_documents(docs)

In [27]:
len(splits)

8

In [28]:
splits[0]

Document(metadata={'source': 'https://www.bbc.com/news/entertainment-arts-68530499', 'title': 'The full list of winners and nominees at the Oscars 2024', 'description': "See who has won and been nominated for this year's coveted Academy Awards in Hollywood.", 'language': 'en-GB', 'start_index': 0}, page_content='The full list of winners and nominees at the Oscars 2024Skip to contentBritish Broadcasting CorporationHomeNewsSportBusinessInnovationCultureArtsTravelEarthVideoLiveHomeNewsIsrael-Gaza WarWar in UkraineUS & CanadaUKUK PoliticsEnglandN. IrelandN. Ireland PoliticsScotlandScotland PoliticsWalesWales PoliticsAfricaAsiaChinaIndiaAustraliaEuropeLatin AmericaMiddle EastIn PicturesBBC InDepthBBC VerifySportBusinessExecutive LoungeTechnology of BusinessFuture of BusinessInnovationTechnologyScience & HealthArtificial IntelligenceAI v the MindCultureFilm & TVMusicArt & DesignStyleBooksEntertainment NewsArtsArts in MotionTravelDestinationsAfricaAntarcticaAsiaAustralia and PacificCaribbean 

In [29]:
splits[1].metadata

{'source': 'https://www.bbc.com/news/entertainment-arts-68530499',
 'title': 'The full list of winners and nominees at the Oscars 2024',
 'description': "See who has won and been nominated for this year's coveted Academy Awards in Hollywood.",
 'language': 'en-GB',
 'start_index': 723}

**3 - Storage**

**Embeddings**

In [30]:
hf_embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
input_test = "just a simple test"
result = hf_embedding.embed_query(input_test)

In [32]:
print(result)

[-0.054198604077100754, -0.09698554128408432, -0.023210447281599045, 0.0071061523631215096, -0.05736067518591881, 0.03741884231567383, 0.03521905839443207, 0.009271848946809769, 0.053111523389816284, 0.03640558570623398, 0.08860345929861069, -0.05440244451165199, 0.020299138501286507, -0.03145493194460869, 0.05896049365401268, 0.010907549411058426, 0.004335182253271341, -0.023187721148133278, 0.012392343021929264, -0.00016039671027101576, -0.041106026619672775, -0.016662200912833214, -0.03658305108547211, -0.02410758100450039, 0.026636894792318344, 0.07495388388633728, 0.0029555719811469316, 0.003283038502559066, 0.008013885468244553, -0.021078625693917274, 0.00030970890657044947, -0.00756420660763979, -0.010757217183709145, -0.026278896257281303, 1.718410544526705e-06, -0.036404259502887726, 0.03562411293387413, 0.0023064131382852793, -0.024003734812140465, 0.026083623990416527, -0.03695273771882057, 0.02897474355995655, -0.030239148065447807, 0.03874632343649864, -0.0234568789601326,

Storing in the vector database

In [33]:
vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embedding)

**Retrieval and Generation Steps**

**4 - Retriever**

In [34]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs = {"k": 6 })

**5 - Generation**

In [35]:
template_rag

"\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nYou are a helpful virtual assistant answering general questions.\nUse the following bits of retrieved context to answer the question.\nIf you don't know the answer, just say you don't know. Keep your answer concise.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nQuestion: {question}\nContext: {context}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n"

In [36]:
prompt_rag = PromptTemplate(input_variables = ["context", "question"], template = template_rag)
prompt_rag

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nYou are a helpful virtual assistant answering general questions.\nUse the following bits of retrieved context to answer the question.\nIf you don't know the answer, just say you don't know. Keep your answer concise.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nQuestion: {question}\nContext: {context}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n")

In [37]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

**Chain**

In [41]:
chain_rag = ({"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt_rag
| llm
| StrOutputParser())

**Generation**

In [42]:
#Test Without RAG
chain.invoke("Which film won the most Oscars at the 2024 awards?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"I'm happy to help! However, I need to correct you - there was no Academy Awards ceremony in 2024. The 97th Academy Awards, honoring the best films of 2023, took place on March 25, 2024. If you're looking for information on past winners or nominees, I'd be happy to provide that for you!"

In [44]:
chain_rag.invoke("Who won the best actor award?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'According to the provided information, Robert Downey Jr. won the Best Supporting Actor award for his role in "Oppenheimer".'

**Cleaning up the vector store**

In [46]:
vectorstore.delete_collection()